# Circuit simulation examples

In [ ]:
import obi_one as obi

In [ ]:
circuit_path_prefix = "/Users/armando/Desktop/Models/microcircuits/cylinder/"

### Loading one circuit

In [ ]:
circuit = obi.Circuit(name="cylinder", path=circuit_path_prefix + "circuit_config.json")
print(f"Circuit '{circuit}' with {circuit.sonata_circuit.nodes.size} neurons and {circuit.sonata_circuit.edges.size} synapses")

### Defining the circuit simulation

In [ ]:
# Simulation init
neuron_set = obi.PredefinedNeuronSet(node_set="All", sample_percentage=None)
sim_duration = 1000.0  # ms
stim_frequency = 20 # Hz
simulations_initialize = obi.CircuitSimulationScanConfig.Initialize(
    circuit=circuit,
    node_set=neuron_set,
    simulation_length=sim_duration,
)

# Stimuli
stim_neuron_set = obi.PredefinedNeuronSet(node_set="All", sample_percentage=100)
stim_times = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=1, interval=1000.0)  # in ms!!
poisson_input = obi.PoissonSpikeStimulus(timestamps=stim_times,
                                         stim_duration=sim_duration,
                                         frequency=stim_frequency,
                                         neuron_set=stim_neuron_set,
                                         )

# Recordings
rec_neuron_set = obi.PredefinedNeuronSet(node_set="All", sample_percentage=100)
v_recording = obi.SomaVoltageRecording(start_time=0.0, end_time=sim_duration, neuron_set=rec_neuron_set)

"""
Fill form with Blocks
"""
simulations_form = obi.CircuitSimulationScanConfig(
                                    initialize=simulations_initialize,
                                    intracellular_location_sets={},
                                    extracellular_locations={},
                                    neuron_sets={"AllNeurons": neuron_set, "AllStim": stim_neuron_set, "AllRec": rec_neuron_set},
                                    synapse_sets={},
                                    timestamps={"StimTimes": stim_times},
                                    stimuli={"PoissonInput": poisson_input},
                                    recordings={"SomaVoltRec": v_recording},
                                    )

In [ ]:
simulations_form.model_dump(mode="json")

In [ ]:
grid_scan = obi.GridScanGenerationTask(form=simulations_form, output_root='../../../obi-output/circuit_simulations/grid_scan')
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
# grid_scan.coordinate_instances(display=True)
grid_scan.execute(processing_method='generate')

In [ ]:
# Deserialization
# grid_scan_ds = obi.deserialize_obi_object_from_json_file("../../../obi-output/circuit_simulations/grid_scan/obi_one_scan.json")